<a href="https://colab.research.google.com/github/peremartra/LLMOptCost/blob/main/00/01_GradioInterface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q gradio
!pip install -q transformers


In [3]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [4]:
# Load the base model and quantized model from Hugging Face
base_model_name = "bigscience/bloomz-560m"
quantized_model_name = "oopere/bloomz-560m-quantized"

In [5]:
# Load the tokenizer (both models use the same tokenizer)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device_map = {"": device} if device != "cpu" else {}

In [9]:
# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                                  device_map=device,
                                                  )


In [10]:
# Load the quantized model
quantized_model = AutoModelForCausalLM.from_pretrained(quantized_model_name,
                                                  device_map=device,
                                                  )

config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [11]:
# Define the function that will take input and pass it to both models
def compare_models(prompt):
    # Prepare input for both models
    inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)

    # Generate output from base model
    base_output = base_model.generate(**inputs)
    base_response = tokenizer.decode(base_output[0], skip_special_tokens=True)

    # Generate output from quantized model
    quantized_output = quantized_model.generate(**inputs)
    quantized_response = tokenizer.decode(quantized_output[0], skip_special_tokens=True)

    # Return both responses
    return base_response, quantized_response

In [12]:
# Create Gradio interface
interface = gr.Interface(
    fn=compare_models,
    inputs=gr.Textbox(label="Input Prompt"),
    outputs=[gr.Textbox(label="Base Model Response"), gr.Textbox(label="Quantized Model Response")],
    title="Compare Base Model vs Quantized Model",
    description="Input a prompt and see how the base and quantized models respond."
)

In [13]:
# Launch the Gradio interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2c1f000a3c815b5d37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
